In [ ]:
# извлечение имени героя и реплики из киносценария

import re
import pandas as pd
import os
from statistics import median
from tqdm import tqdm_notebook
from collections import Counter

def get_the_hero_and_lines(folder_input,folder_output):
    folder = os.listdir(folder_input)
    for filename in tqdm_notebook(folder):
        if 'txt' in filename:
            with open(folder_input+filename, encoding="utf8", errors='ignore') as f:
                text = f.read()
            lines = text.split('\n')

            replics = []
            small_let = re.compile('[a-z]')
            dig = re.compile('[\d]')
            stoplist = ["THE END.", "CONTINUED", "INT.", "EXT.", "FADE IN", "FADE OUT", "CUT TO:", 'DISSOLVE', "DRAFT", "MED", "CLOSE", "FULL", "REVISED"]
            
            
            replic = False

            for i, line in enumerate(lines):
                indent = (len(line) - len(line.lstrip()))

                # Если имя нашлось, мы в режиме "реплика"     
                if replic:
                    if not line.strip().startswith('(') and not line.strip().endswith(')') and len(line.strip()) > 0\
                        and not (re.search(dig, line) and len(line.strip())<3) and not stop_word:
                        if not first_phrase:
                            if indent == last_indent:
                                phrase += ' ' + line.strip().strip('*').strip()
                            else:
                                replic = False
                                phrase = phrase
                                replics.append([name, phrase])
                        else:
                            phrase = line.strip().strip('*').strip()
                        last_indent = indent
                        first_phrase = False

                # Условие "если содержит '(', то содержит и ')'    
                bad_line = False
                if ('(' in line and not ')' in line) or \
                    (')' in line and not '(' in line):
                    bad_line = True
                        
                #   Стоп-слова
                stop_word = False
                for word in stoplist:
                    if word in line:
                        stop_word = True
                        break

                # Поиск имени    
                if indent > 19 and indent < 40 and not bad_line and not line.strip().startswith('(') \
                and not re.search(small_let, line) and not stop_word \
                and not re.search(dig, line):
                    testname = line.strip().strip('*').strip()
                    
                    if len(testname) > 0:
                        name = testname
                        replic = True
                        first_phrase = True
                    

            df = pd.DataFrame(replics, columns = ['name', 'phrase'])
            df.dropna().to_csv(folder_output+filename.replace('.txt','')+".csv", index=False)



In [ ]:
#извлекаем медианную длину реплики персонажа
def get_the_medium_len__phrase(folder_input,folder_output):
    hero_len_line = {}
    the_end_file = []
    film_mean_line = []
    folder = os.listdir(folder_input)
    for filename in folder:
        if '.csv'in filename:
            with open(folder_input+filename, encoding="utf8", errors='ignore') as f:
                df=pd.read_csv(f)
                replics = df.dropna().values.tolist()
            for element in replics:
                if element[0].isdigit()==False:#не считаем  реплику с цифрой в имени
                    if element[0].strip():  #не считаем  пустое имя героя
                        if not element[1].replace('.','').isdigit(): #елси реплика только цифра 
                            length = len(element[1].replace("-","").replace("*","").replace("..."," ").split())
                            the_file = filename.replace(".csv","").replace(".txt","").replace("txt","").strip()
                            if the_file not in hero_len_line:
                                hero_len_line[the_file] = []
                            hero_len_line[the_file].append(length)
    for key, value in hero_len_line.items():
        the_title = key.split(";")
        the_title.append(median(value))
        the_title.append(max(value))
        the_end_file.append(the_title)
    data = pd.DataFrame(the_end_file,columns = ["year","genre","title","mean length","max length"])
    data.to_csv(folder_output+'_mean_length.csv', index=True)


In [ ]:
#извлекаем количество персонажей в сценарии 

import re
def extract_hero(line):
    m = re.match("( *)([A-Z\.]{2,}[A-Z \.]*)", line)
    if not m:
        return
    indent = len(m.group(1))
    if indent > 19 and indent < 45:
        return m.group(2).strip()
    
def get_the_number_of_heroes(folder_input,folder_output):
    list_film_heroes = []
    film_num_heroes = []
    film_and_heroes = []
    folder = os.listdir(folder_input)
    stoplist = set(['"THE END", "CONTINUED", "INT.", "EXT.", "FADE IN", "FADE OUT", "CUT TO","DISSOLVE TO"," EARLY DRAFT"'
                   '"CLOSEUP","CLOSE UP","WIDE VIEW","LONG SHOT","MEDIUM","MEDIUM SHOT","MEDIUM SHOT.",'
                   '"MEDIUM CLOSE SHOT","MEDIUM CLOSEUP","MED.CLOSE SHOT","MEDIUM", "LONG SHOT","MED. LONG SHOT",'
                   '"EXTREME CLOSE UP","EXTREME CLOSE","EXTREME CLOSEUPS","PAN RIGHT","PAN LEFT","PAN UP","PAN DOWN",'
                   '"TILT UP","TILT DOWN","TILT RIGHT","TILT LEFT","POINT OF VIEW","SLOW MOTION","FAST MOTION"'
                   '"SLO-MO","ZOOM OUT","ZOOM BACK","ZOOM IN"'])
    
    for filename in folder:
        if '.txt' in filename:
            with open(folder_input+filename, encoding="utf8", errors='ignore') as f:
                my_heroes = {}
                for line in f:
                    hero = extract_hero(line)
                    if (hero and hero not in stoplist and 
                        hero!=f.name.split("/")[-1].replace(".txt","").strip().upper()):
                            my_heroes[hero] = 0
                list_film_heroes.append((filename.replace(".txt",""),my_heroes))

    for element in list_film_heroes:
        name  = element[0].split(";") 
        heroes = list(element[1])
        film_and_heroes.append((name,heroes))
        
    for i,element in enumerate(film_and_heroes):
        name_of_h = element[0]
        length = len(element[1])
        name_of_h.append(length)
        film_num_heroes.append(name_of_h)

    df=pd.DataFrame(film_num_heroes, columns = ('year', 'genre','film title', 'number of heroes'))
    df.to_csv(folder_output+'_how_many_heroes_in_film.csv',index =True)

In [ ]:
#извлекаем количество плана и вида съемки 



def extract_the_shot(line):
    reg = re.search("\s*(CU)\s+.*|\s*(C.U.)\s+.*|\s*(WIDE)\s+.*|\s*(POV)\s+.*|\s*(FM)\s+.*|\s*(SM)\s+.*|",line)
    if reg:
        return reg.group()

def extract_num_of_shots(folder_input):
    words = ["CU","WIDE","C.U.","POV", "FM","SM"]
    film_here= []
    lines_reg = []
    film_shots = {}
    new_film_shot = {}
    d_film_shot = {}
    the_list_with_shots =[]
    data_of_filmshots = []
    folder = os.listdir(folder_input)
    for filename in folder:
        if '.txt' in filename:
            with open(folder_input+filename, encoding="utf8", errors='ignore') as f:
                for line in f:
                    shot= extract_the_shot(line)
                    if shot:
                        if filename not in film_shots:
                            film_shots[filename]= []
                        film_shots[filename].append(shot)
                for key,value in film_shots.items():
                    film_here.append(key)
                    lines_reg = list(value)
                for element in lines_reg:
                    for i in words:
                        if i in element:
                            if filename not in new_film_shot:
                                new_film_shot[filename] = []
                            new_film_shot[filename].append(i)
        for key,value in new_film_shot.items():
            new_film_shot[key]=Counter(value)
            d_film_shot = new_film_shot
    for key, value in d_film_shot.items():
        the_list_with_shots.append(key.replace('.txt',"").split(";"))
        data_of_filmshots.append(value)
    film_data = pd.DataFrame(the_list_with_shots, columns = ["year", "genre", "title"])
    shot_data = pd.DataFrame(data_of_filmshots)
    data_f= pd.concat([film_data,shot_data],axis=1)
    return data_f 


def extract_shot(line):
    r_shot = re.search('CLOSEUP|CLOSE UP|WIDE VIEW|LONG SHOT|MEDIUM|MEDIUM SHOT|'
                    'MEDIUM SHOT.|MEDIUM CLOSE SHOT|MEDIUM CLOSEUP|MED.CLOSE SHOT|'
                    'MEDIUM LONG SHOT|MED. LONG SHOT|EXTREME CLOSE UP|EXTREME CLOSE|'
                    'EXTREME CLOSEUPS|PAN RIGHT|PAN LEFT|PAN UP|PAN DOWN|TILT UP|'
                    'TILT DOWN|TILT RIGHT|TILT LEFT|POINT OF VIEW|SLOW MOTION|FAST MOTION|'
                    'SLO-MO|ZOOM OUT|ZOOM BACK|ZOOM IN',line) 
    if r_shot:
        return r_shot.group()

def extract_number_of_shots(folder_input):
    film_shots = {}
    d_film_shot = {}
    the_list_with_title =[]
    data_of_filmshots = []
    folder = os.listdir(folder_input)
    for filename in folder:
        if '.txt' in filename:
            with open(folder_input+filename, encoding="utf8", errors='ignore') as f:
                for line in f:
                    shot= extract_shot(line)
                    if shot:
                        if filename not in film_shots:
                            film_shots[filename]= []
                        film_shots[filename].append(shot)
        for key,value in film_shots.items():
            film_shots[key]=Counter(value)
            d_film_shot = film_shots
    for key, value in d_film_shot.items():
        the_list_with_title.append(key.replace('.txt',"").split(";"))
        data_of_filmshots.append(value)
    film_data = pd.DataFrame(the_list_with_title, columns = ["year", "genre", "title"])
    shot_data = pd.DataFrame(data_of_filmshots)
    df= pd.concat([film_data,shot_data],axis="columns")
    return df

def add_file():
    folder_input =''
    folder_output =''
    first_file = extract_num_of_shots(folder_input)
    second_file = extract_number_of_shots(folder_input)
    new = second_file.merge(first_file, on= ("genre","title","year"))
    new.to_csv(folder_output+'_number_of_shots.csv')

